In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn 

In [ ]:
a=[0,1,2,3,4,5]
b=[34,23,34,45,56,45]

print(f'\n zip converts list to tuple ')
for x in zip(a, b):
    print(x,type(x))

print(f'\n zip converts list to int by one by')
for x,y in zip(a, b):
    print(x,type(x))

print(f'\n zip slice and converts list to tuple')
for x in zip(a,a[1:3]):
    print(x,type(x))

print(f'\n zip slice and converts list to int')
for x,y in zip(a,a[1:3]):
    print(x,type(x))

In [24]:
print(x)